# Databases and MySQL

In [ ]:
# Setting Up
import mysql.connector
# if import fails, 'conda install mysql-connector-python' to install (from a conda prompt)
import pandas as pd

## Flight-Airplane-PassengerCount

In [ ]:
# read from csv
flights = pd.read_csv("../data/flights.csv", parse_dates=['DepDate', 'ArrDate'])
flights.head()

## People-Flight-Airplane

In [ ]:
# read from csv
passengers = pd.read_csv("../data/passengers.csv", parse_dates=['DepDate', 'ArrDate'])
passengers.head()

## Connecting to and querying the database

In [ ]:
# connect to the database
cnx = mysql.connector.connect(
    user="insy6500", 
    password="insy6500", 
    host="localhost", 
    database="insy6500"
    )
print ("Successfully connected....")

In [ ]:
# List the tables using a cursor object
print ("Tables:")
cursor = cnx.cursor()
# this is the SQL query
query = ("show tables")
cursor.execute(query)
for (db) in cursor:
    print ("\t{}".format(db[0]))
cursor.close()
print ("Database closed.")

## Define Some Helper Functions

In [ ]:
#
#  Single value query - returns the result of a single-value query
#
def single_value(query):
    cursor = cnx.cursor()
    cursor.execute(query)
    db = cursor.fetchall()
    cursor.close()
    return(db[0][0])

#
#  Recordset query - returns the result of a recordset query
#
def recordset(query):
    cursor = cnx.cursor()
    cursor.execute(query)
    db = cursor.fetchall()
    cursor.close()
    return(db)

In [ ]:
qry = "select count(*) from passenger"
print("{:,d} passenger records.".format(single_value(qry)))

In [ ]:
qry = "select * from airplane"
print("Airplane records: {}".format(recordset(qry)))

In [ ]:
rs = recordset(qry)

In [ ]:
for r in rs:
    print(r[0])

## Using Pandas

In [ ]:
# Sample query to a Pandas dataframe
qry = "select * from airplane"
print ("Query: {:}".format(qry))
pdPlanes = pd.read_sql(qry, cnx)
pdPlanes

In [ ]:
# Query to an anonymous dataframe
pd.read_sql("show tables", cnx)

In [ ]:
qry = "select * from passenger limit 5"
pd.read_sql(qry, cnx)

In [ ]:
# Define a query helper function
# Assumes that the connection cnx has already been established
def pquery(qry):
    p = pd.read_sql(qry,cnx)
    print("Found {:,d} records.".format(len(p)))
    return p

In [ ]:
qry = "select * from booking limit 25"
p = pquery(qry)
p.head()

In [ ]:
# Example with a more complex query.
qry = '''
SELECT 
    flightnumber, 
    origin, 
    registrationnumber, 
    capacity, 
    count(emailaddress) as bookings, 
    capacity - count(emailaddress) AS remaining 
FROM 
    flight 
    INNER JOIN airplane USING (registrationnumber) 
    INNER JOIN booking USING (flightnumber)
WHERE 
    origin = 'Atlanta' GROUP BY flightnumber HAVING remaining > 0;
'''
p = pquery(qry)
p.head()